In [64]:
%run database.py
import pandas as pd

In [56]:
conn = create_connection()

Connection to SQLite DB successful


In [57]:
# define columns as a list of pairs with title and type
columns = [
    ('HiddenService', 'TEXT PRIMARY KEY'), # TODO: Make FK in other tables?
]

In [58]:
tablename = 'services'

In [66]:
#drop the table from the connection
drop_table(tablename, conn)

Table services dropped


In [67]:
conn.execute(
    generateCreateTableQuery(tablename, columns)
)

CREATE TABLE IF NOT EXISTS "services" (
"HiddenService" TEXT PRIMARY KEY);


In [68]:
# get all the hidden services from the table pings and store them into a dataframe
df = pd.read_sql_query("SELECT DISTINCT HiddenService FROM pings", conn)
# make HiddenService the index
df.set_index('HiddenService', inplace=True)

In [69]:
df

""
HiddenService
*youbroketheinternet@loupsycedyglgamf.onion
1111111111111111.111111111111111111111111111.1w2e3r456y7u1q.onion
12345denis.3voaeq7o2obsrocx.onion
12345denis.sholq4kbukl5mitc.onion
12345denis.shops3jqx4hojok2.onion
...
zzz7hcycjoj7mqc6.onion
zzzatqyocf7kt7y6.onion
zzzcgjit65yyn4ji.onion


In [70]:
# insert the hidden services into the table services
# don't insert the rowid
df.to_sql(tablename, conn, if_exists='append', index=True, index_label='HiddenService')

20487

In [31]:
close_connection(conn)

The SQLite connection is closed
